# Scenario Selection Evaluation

This notebook validates the selected scenarios, plots representative graphs, and computes clustering metrics (e.g., silhouette score)


In [15]:
from pathlib import Path
import json
import os
import glob
import pickle
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score, silhouette_samples
import warnings

warnings.filterwarnings('ignore')


def find_repo_root(start=None):
    if start is None:
        start = Path.cwd()
    start = start.resolve()
    for parent in [start] + list(start.parents):
        if (parent / '.git').exists():
            return parent
        if (parent / 'data' / 'sub20' / 'graphs').exists():
            return parent
    raise FileNotFoundError("Could not find repo root (no .git or data/sub20/graphs found)")


ROOT = find_repo_root()

# Prefer repo-root graphs/ if it exists, otherwise fall back to data/sub20/graphs
if (ROOT / 'graphs').exists():
    GRAPHS_DIR = ROOT / 'graphs'
else:
    GRAPHS_DIR = ROOT / 'data' / 'sub20' / 'graphs'

OUTPUT_DIR = ROOT / 'src' / 'kernels' / 'outputs'
OUTPUT_DIR.mkdir(exist_ok=True)

print('ROOT:', ROOT)
print('GRAPHS_DIR:', GRAPHS_DIR)
print('OUTPUT_DIR:', OUTPUT_DIR)



ROOT: /Users/ignaciagothe/Desktop/proyecto graph kernels/graph_scenarios
GRAPHS_DIR: /Users/ignaciagothe/Desktop/proyecto graph kernels/graph_scenarios/data/sub20/graphs
OUTPUT_DIR: /Users/ignaciagothe/Desktop/proyecto graph kernels/graph_scenarios/src/kernels/outputs


In [16]:
# Load selection outputs
K_VALUES = [20, 100]


def load_selection(k):
    path_graphs = OUTPUT_DIR / f"selected_graphs_k{k}.json"
    path_scenarios = OUTPUT_DIR / f"selected_scenarios_k{k}.json"
    if path_graphs.exists():
        data = json.loads(path_graphs.read_text())
        data['_path'] = path_graphs
        data['_format'] = 'selected_graphs'
        return data
    if path_scenarios.exists():
        data = json.loads(path_scenarios.read_text())
        data['_path'] = path_scenarios
        data['_format'] = 'selected_scenarios'
        return data
    raise FileNotFoundError(f"No selection file found for k={k} in {OUTPUT_DIR}")


selections = {k: load_selection(k) for k in K_VALUES}
for k, data in selections.items():
    print(f"k={k} | format={data['_format']} | path={data['_path'].name}")


k=20 | format=selected_graphs | path=selected_graphs_k20.json
k=100 | format=selected_graphs | path=selected_graphs_k100.json


In [17]:
# Build graph file index (for validation)
pattern = None
for data in selections.values():
    if 'pattern' in data:
        pattern = data['pattern']
        break
if pattern is None:
    pattern = 'graph_*.pickle'

all_graph_files = sorted(glob.glob(str(GRAPHS_DIR / pattern)))
all_graph_basenames = [os.path.basename(p) for p in all_graph_files]
name_to_index = {name: idx for idx, name in enumerate(all_graph_basenames)}

print(f"Found {len(all_graph_files)} graphs using pattern '{pattern}'")


Found 49125 graphs using pattern 'graph_*.pickle'


In [18]:
# Parse selection outputs into a common shape and validate file selection

def parse_selection(data, k):
    selected_files = []
    selected_indices = None
    cluster_labels = None
    cluster_sizes = None

    if data['_format'] == 'selected_graphs':
        selected_files = data['selected_pickle_files']
        selected_indices = data.get('selected_indices')
        cluster_labels = np.array(data.get('cluster_labels', []), dtype=int)
        cluster_sizes = data.get('cluster_sizes')
    else:
        selected_files = [s['filename'] for s in data['selected_scenarios']]
        cluster_sizes = [s['cluster_size'] for s in data['selected_scenarios']]
        labels_path = OUTPUT_DIR / f"cluster_assignments_k{k}.npy"
        if labels_path.exists():
            cluster_labels = np.load(labels_path)

    # Validate selected files exist
    missing = [f for f in selected_files if f not in name_to_index]
    if missing:
        print(f"[warn] k={k}: missing files: {missing[:5]}{'...' if len(missing) > 5 else ''}")
    else:
        print(f"[ok] k={k}: all selected files exist")

    # Validate index mapping if provided
    if selected_indices is not None:
        mismatches = []
        for idx, fname in zip(selected_indices, selected_files):
            if idx >= len(all_graph_basenames) or all_graph_basenames[idx] != fname:
                mismatches.append((idx, fname))
        if mismatches:
            print(f"[warn] k={k}: index->filename mismatches: {mismatches[:3]}")
        else:
            print(f"[ok] k={k}: selected_indices match filenames")

    # Validate cluster sizes if labels are present
    label_counts = None
    if cluster_labels is not None and cluster_labels.size > 0:
        label_counts = np.bincount(cluster_labels, minlength=k)
        if cluster_sizes is not None and len(cluster_sizes) == k:
            if not np.all(label_counts == np.array(cluster_sizes)):
                print(f"[warn] k={k}: cluster_sizes do not match label counts")
            else:
                print(f"[ok] k={k}: cluster_sizes match label counts")
        if selected_indices is not None:
            # Medoid label should match its cluster id (order of medoids)
            bad_medoids = []
            for cluster_id, medoid_idx in enumerate(selected_indices):
                if medoid_idx < len(cluster_labels) and cluster_labels[medoid_idx] != cluster_id:
                    bad_medoids.append((cluster_id, medoid_idx, cluster_labels[medoid_idx]))
            if bad_medoids:
                print(f"[warn] k={k}: medoid label mismatches: {bad_medoids[:3]}")
            else:
                print(f"[ok] k={k}: medoid labels consistent")

    return {
        'selected_files': selected_files,
        'selected_indices': selected_indices,
        'cluster_labels': cluster_labels,
        'cluster_sizes': cluster_sizes,
        'label_counts': label_counts,
    }


parsed = {k: parse_selection(data, k) for k, data in selections.items()}


[ok] k=20: all selected files exist
[ok] k=20: selected_indices match filenames
[ok] k=20: cluster_sizes match label counts
[ok] k=20: medoid labels consistent
[ok] k=100: all selected files exist
[ok] k=100: selected_indices match filenames
[ok] k=100: cluster_sizes match label counts
[ok] k=100: medoid labels consistent


In [19]:
# --- WL features for full silhouette computation ---
import sys
from scipy import sparse
from tqdm import tqdm

kernels_dir = ROOT / 'src' / 'kernels'
if str(kernels_dir) not in sys.path:
    sys.path.append(str(kernels_dir))

from kernel_baseline import DirectedWLKernelHasher, row_l2_norms, row_normalize, load_nx_dag

# WL params from selection metadata (fallback defaults)
meta = selections[K_VALUES[0]]
wl_iterations = meta.get('wl_iterations', 3)
hash_dim = meta.get('hash_dim', 2**18)
node_label_attr = meta.get('node_label_attr')

X_cache = OUTPUT_DIR / f"wl_features_h{wl_iterations}_d{hash_dim}.npz"
if X_cache.exists():
    X = sparse.load_npz(X_cache)
    print(f"Loaded cached WL features: {X_cache} | shape={X.shape} nnz={X.nnz}")
else:
    hasher = DirectedWLKernelHasher(h=wl_iterations, hash_dim=hash_dim, node_label_attr=node_label_attr)
    rows, cols, data = [], [], []
    for i, path in enumerate(tqdm(all_graph_files, desc='Computing WL features')):
        G = load_nx_dag(path)
        fd = hasher._wl_features_one_graph(G)
        for c, v in fd.items():
            rows.append(i)
            cols.append(c)
            data.append(v)

    X = sparse.csr_matrix((data, (rows, cols)), shape=(len(all_graph_files), hash_dim), dtype=np.float64)
    sparse.save_npz(X_cache, X)
    print(f"Saved WL features: {X_cache} | shape={X.shape} nnz={X.nnz}")

norms = row_l2_norms(X)
Xn = row_normalize(X, norms)
print('Xn ready:', Xn.shape, 'nnz', Xn.nnz)



Loaded cached WL features: /Users/ignaciagothe/Desktop/proyecto graph kernels/graph_scenarios/src/kernels/outputs/wl_features_h3_d262144.npz | shape=(49125, 262144) nnz=19817686
Xn ready: (49125, 262144) nnz 19817686


In [ ]:
# --- Full silhouette scores (no sampling) ---
from scipy import sparse


def full_silhouette_samples_from_Xn(Xn, labels, chunk_size=100):
    labels = np.asarray(labels, dtype=int)
    n = Xn.shape[0]
    k = int(labels.max()) + 1
    cluster_sizes = np.bincount(labels, minlength=k).astype(np.int64)
    if np.any(cluster_sizes == 0):
        raise ValueError('Empty cluster detected in labels')

    membership = sparse.csr_matrix(
        (np.ones(n, dtype=np.float64), (np.arange(n), labels)),
        shape=(n, k)
    )

    sil_samples = np.empty(n, dtype=np.float32)

    for start in tqdm(range(0, n, chunk_size), desc='Silhouette (full)'):
        end = min(start + chunk_size, n)
        Xc = Xn[start:end]

        # Similarity to all points
        S = (Xc @ Xn.T).toarray()
        np.clip(S, -1.0, 1.0, out=S)

        # Convert similarity to Euclidean distance on unit sphere: sqrt(2 - 2*cos)
        S *= -2.0
        S += 2.0
        np.maximum(S, 0.0, out=S)
        np.sqrt(S, out=S)  # now distances

        sums = S @ membership  # (m, k) sum distances to each cluster
        labels_chunk = labels[start:end]
        sizes = cluster_sizes[labels_chunk]

        sums_own = sums[np.arange(end - start), labels_chunk]
        a = np.zeros(end - start, dtype=np.float64)
        mask = sizes > 1
        a[mask] = sums_own[mask] / (sizes[mask] - 1)

        mean_other = sums / cluster_sizes
        mean_other[np.arange(end - start), labels_chunk] = np.inf
        b = np.min(mean_other, axis=1)

        s = (b - a) / np.maximum(a, b)
        s[~np.isfinite(s)] = 0.0
        sil_samples[start:end] = s.astype(np.float32)

    return sil_samples


full_silhouette = {}
for k in K_VALUES:
    labels = parsed[k]['cluster_labels']
    if labels is None or len(labels) == 0:
        print(f'k={k}: no cluster labels available for silhouette')
        continue
    if len(labels) != len(all_graph_files):
        print(f'k={k}: label length {len(labels)} != n_graphs {len(all_graph_files)}, skipping')
        continue

    sil = full_silhouette_samples_from_Xn(Xn, labels, chunk_size=100)
    full_silhouette[k] = sil
    score = float(np.mean(sil))
    print(f'k={k} full silhouette: {score:.4f}')

    np.save(OUTPUT_DIR / f"silhouette_samples_k{k}.npy", sil)



Silhouette (full):  37%|███▋      | 182/492 [01:22<02:23,  2.17it/s]

In [ ]:
# Cluster size distributions
for k in K_VALUES:
    label_counts = parsed[k]['label_counts']
    if label_counts is None:
        print(f"k={k}: no cluster labels available for size plots")
        continue

    fig, ax = plt.subplots(1, 1, figsize=(10, 4))
    ax.bar(range(k), np.sort(label_counts)[::-1], color='steelblue', edgecolor='black')
    ax.set_title(f'Cluster Sizes (k={k}) | min={label_counts.min()}, mean={label_counts.mean():.1f}, max={label_counts.max()}')
    ax.set_xlabel('Cluster (sorted by size)')
    ax.set_ylabel('Cluster size')
    ax.axhline(label_counts.mean(), color='red', linestyle='--', label='Mean')
    ax.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
# --- Silhouette & cluster visualizations (full) ---
for k in K_VALUES:
    if k not in full_silhouette:
        continue

    sil = full_silhouette[k]
    labels = np.asarray(parsed[k]['cluster_labels'], dtype=int)
    cluster_sizes = np.bincount(labels, minlength=k)
    cluster_means = np.array([
        sil[labels == c].mean() if cluster_sizes[c] > 0 else np.nan
        for c in range(k)
    ])

    # 1) Silhouette distribution
    fig, ax = plt.subplots(1, 1, figsize=(8, 4))
    ax.hist(sil, bins=50, color='teal', edgecolor='black', alpha=0.7)
    ax.axvline(sil.mean(), color='red', linestyle='--', label=f"Mean: {sil.mean():.4f}")
    ax.set_title(f"Silhouette Distribution (k={k})")
    ax.set_xlabel('Silhouette coefficient')
    ax.set_ylabel('Frequency')
    ax.legend()
    ax.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

    # 2) Mean silhouette per cluster (sorted)
    order = np.argsort(cluster_means)[::-1]
    fig, ax = plt.subplots(1, 1, figsize=(10, 4))
    ax.bar(range(k), cluster_means[order], color='steelblue', edgecolor='black')
    ax.set_title(f"Mean Silhouette by Cluster (k={k})")
    ax.set_xlabel('Cluster (sorted by mean silhouette)')
    ax.set_ylabel('Mean silhouette')
    ax.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

    # 3) Cluster size vs mean silhouette
    fig, ax = plt.subplots(1, 1, figsize=(6, 4))
    ax.scatter(cluster_sizes, cluster_means, alpha=0.6)
    ax.set_title(f"Cluster Size vs Mean Silhouette (k={k})")
    ax.set_xlabel('Cluster size')
    ax.set_ylabel('Mean silhouette')
    ax.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

    # 4) Cumulative coverage
    weights = cluster_sizes / cluster_sizes.sum()
    cumulative = np.cumsum(np.sort(weights)[::-1])
    fig, ax = plt.subplots(1, 1, figsize=(6, 4))
    ax.plot(range(1, k + 1), cumulative * 100, linewidth=2)
    ax.axhline(50, color='red', linestyle='--', alpha=0.5, label='50%')
    ax.axhline(80, color='orange', linestyle='--', alpha=0.5, label='80%')
    ax.set_title(f"Cumulative Coverage (k={k})")
    ax.set_xlabel('Number of clusters')
    ax.set_ylabel('Coverage (%)')
    ax.legend()
    ax.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()



In [ ]:
def load_graph_by_name(filename):
    with open(GRAPHS_DIR / filename, 'rb') as f:
        return pickle.load(f)


def graph_stats(G):
    return {
        'nodes': G.number_of_nodes(),
        'edges': G.number_of_edges(),
        'density': nx.density(G) if G.number_of_nodes() > 1 else 0.0,
    }


def grid_positions_20(G):
    nodes = list(G.nodes())
    if not nodes:
        return None
    if not all(isinstance(n, (int, np.integer)) for n in nodes):
        return None
    min_n = int(min(nodes))
    max_n = int(max(nodes))
    if 0 <= min_n and max_n <= 399:
        offset = 0
    elif 1 <= min_n and max_n <= 400:
        offset = 1
    else:
        return None

    pos = {}
    for node in nodes:
        idx = int(node) - offset
        row = idx // 20
        col = idx % 20
        pos[node] = (col, 19 - row)
    return pos


def draw_fire_spread_graph(G, ax, title=''):
    pos = grid_positions_20(G)
    if pos is None:
        pos = nx.spring_layout(G, seed=42)

    if G.is_directed():
        ignition_nodes = [n for n in G.nodes() if G.in_degree(n) == 0]
    else:
        ignition_nodes = []

    node_colors = ['red' if n in ignition_nodes else 'orange' for n in G.nodes()]

    nx.draw_networkx_nodes(G, pos, ax=ax, node_size=30, node_color=node_colors, alpha=0.8)
    nx.draw_networkx_edges(G, pos, ax=ax, edge_color='gray', alpha=0.5,
                           arrows=G.is_directed(), arrowsize=5, width=0.5)

    ax.set_title(title, fontsize=8)
    ax.set_axis_off()


def plot_selected_graphs(k):
    selected_files = parsed[k]['selected_files']
    if not selected_files:
        print(f"k={k}: no selected files to plot")
        return

    graphs = []
    for fname in selected_files:
        G = load_graph_by_name(fname)
        stats = graph_stats(G)
        graphs.append((fname, G, stats))

    cols = 5
    rows = int(np.ceil(len(graphs) / cols))
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 3.2, rows * 3.0))
    axes = np.array(axes).reshape(-1)

    for i, (fname, G, stats) in enumerate(graphs):
        title = f"{fname}\n{stats['nodes']}n, {stats['edges']}e"
        draw_fire_spread_graph(G, axes[i], title)

    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    plt.suptitle(f"Selected Graphs (k={k})", fontsize=12)
    plt.tight_layout()
    plt.show()


plot_selected_graphs(20)

In [ ]:
plot_selected_graphs(100)